In [ ]:
import sys
sys.path.append("./../")

In [ ]:
from pathlib import Path
import math
import pickle
#
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
#
from misc.plot_utils import plot_mat

In [ ]:
p_experiments = Path("/mnt/experiments/effcn/affnist/grid_search_wob/")
p_experiments.exists()

# ALL RUNS

In [ ]:
runs_nan = []
runs_success = []
for p_experiment in p_experiments.iterdir():
    p_config = p_experiment / "config.pkl"
    p_stats = p_experiment / "stats.pkl"
    if not p_stats.exists():
        continue
    with open(p_config, "rb") as file:
        config = pickle.load(file)
    with open(p_stats, "rb") as file:
        stats = pickle.load(file)
        all_accs = stats["valid"]["affnist"]["acc"]
        if math.isnan(stats["train"]["loss"][-1]):
            runs_nan.append((config, stats))
        else:
            runs_success.append((config, stats))
        max_acc = max(all_accs)
        #print("bs={:4d}, lr={:.5f}, wd={:.5f} rec_weights={:8.5f}  max_acc={:.4f} notes={}".format(
        #    config.train.batch_size,
        #    config.optimizer_args.lr,
        #    config.optimizer_args.weight_decay,
        #    config.loss.rec.weight,
        #    max_acc,
        #    stats["notes"]
        #     )
        #     )

In [ ]:
print("#success:      ", len(runs_success))
print("#failed (nan): ", len(runs_nan))

# Check failed runs

In [ ]:
for config, stats in runs_nan:
    all_accs = stats["valid"]["affnist"]["acc"]
    max_acc = max(all_accs)
    print("bs={:4d}, lr={:.5f}, wd={:.5f} rec_weights={:8.5f}  max_acc={:.4f} notes={}".format(
            config.train.batch_size,
            config.optimizer_args.lr,
            config.optimizer_args.weight_decay,
            config.loss.rec.weight,
            max_acc,
            stats["notes"]
             )
             )

In [ ]:
data = []
for config, stats in runs_nan:
    lr = config.optimizer_args.lr
    bs = config.train.batch_size
    wd = config.optimizer_args.weight_decay
    rl = config.loss.rec.weight
    #
    data.append([lr, bs, wd, rl, 1.0])
for config, stats in runs_success:
    lr = config.optimizer_args.lr
    bs = config.train.batch_size
    wd = config.optimizer_args.weight_decay
    rl = config.loss.rec.weight
    #
    data.append([lr, bs, wd, rl, 0.0])
data = np.array(data)
names = ["lr", "bs", "weight_decay", "rec_loss", "nan in loss"]

In [ ]:
C = np.corrcoef(data.T)
plot_mat(C, row_names=names, col_names=names, scale_factor=1, title="Effect of parameters on Training Success or Failure due to NaN")

Results:
- failed when LR was high and weight decay was small and batch size was small

# Check successruns with high lr

In [ ]:
for config, stats in runs_success:
    if config.optimizer_args.lr != 1e-2:
        continue
    all_accs_an = stats["valid"]["affnist"]["acc"]
    all_accs_mn = stats["valid"]["mnist"]["acc"]
    max_acc_an = max(all_accs_an)
    max_acc_mn = max(all_accs_mn)
    print("bs={:4d}, lr={:.5f}, wd={:.5f} rec_weights={:8.5f}  max_acc_aff={:.4f} max_acc_mn={:.4f}".format(
            config.train.batch_size,
            config.optimizer_args.lr,
            config.optimizer_args.weight_decay,
            config.loss.rec.weight,
            max_acc_an,
            max_acc_mn
             )
             )

Results:
- weight decay can compensate large learning rates

# All Time Highs

In [ ]:
for config, stats in runs_success:
    all_accs_an = stats["valid"]["affnist"]["acc"]
    all_accs_mn = stats["valid"]["mnist"]["acc"]
    max_acc_an = max(all_accs_an)
    max_acc_mn = max(all_accs_mn)
    if max_acc_an < 0.87:
        continue
    print("bs={:4d}, lr={:.5f}, wd={:.5f} rec_weights={:8.5f}  max_acc_aff={:.4f} max_acc_mn={:.4f}".format(
            config.train.batch_size,
            config.optimizer_args.lr,
            config.optimizer_args.weight_decay,
            config.loss.rec.weight,
            max_acc_an,
            max_acc_mn
             )
             )
    print(p_experiments / config.names.model_dir)

In [ ]:
p_experiments

# Compare effect of WD vs NWD on ACC

In [ ]:
# with weight decay vs without weight decay
acc_vals = {}
for config, stats in runs_success:
    all_accs = stats["valid"]["affnist"]["acc"][-50:]
    acc_mean = np.mean(all_accs)
    acc_min = np.min(all_accs)
    acc_max = np.max(all_accs)
    acc_var = np.var(all_accs)
    wd = config.optimizer_args.weight_decay
    if wd not in acc_vals:
        acc_vals[wd] = []
    acc_vals[wd].append((acc_mean, acc_min, acc_max, acc_var))

In [ ]:
wd_acc = [(wd, np.array(vals).mean(axis=0)) for wd, vals in acc_vals.items()]
wd_acc = sorted(wd_acc, key=lambda x: x[0])
for wd, vals in wd_acc:
    print("wd={:.5f}: mean={:.4f} min={:.4f} max={:.4f} var={:.8f}".format(wd, vals[0], vals[1], vals[2], vals[3]))

results:
- weight decay stabilizes training
- weight decay results in better accuracy

# Compare effect of REC vs NO REC on ACC

In [ ]:
# with weight decay vs without weight decay
acc_vals = {}
for config, stats in runs_success:
    all_accs = stats["valid"]["affnist"]["acc"][-30:]
    acc_mean = np.mean(all_accs)
    acc_min = np.min(all_accs)
    acc_max = np.max(all_accs)
    acc_var = np.var(all_accs)
    rec = config.loss.rec.weight
    if rec not in acc_vals:
        acc_vals[rec] = []
    acc_vals[rec].append((acc_mean, acc_min, acc_max, acc_var))

In [ ]:
rec_acc = [(rec, np.array(vals).mean(axis=0)) for rec, vals in acc_vals.items()]
rec_acc = sorted(rec_acc, key=lambda x: x[0])
for rec, vals in rec_acc:
    print("rec={:.5f}: mean={:.4f} min={:.4f} max={:.4f} var={:.8f}".format(rec, vals[0], vals[1], vals[2], vals[3]))

In [ ]:
# with weight decay vs without weight decay
acc_vals = {}
for config, stats in runs_success:
    all_accs = stats["valid"]["affnist"]["acc"][-30:]
    if max(all_accs) < 0.80:
        continue
    acc_mean = np.mean(all_accs)
    acc_min = np.min(all_accs)
    acc_max = np.max(all_accs)
    acc_var = np.var(all_accs)
    rec = config.loss.rec.weight
    if rec not in acc_vals:
        acc_vals[rec] = []
    acc_vals[rec].append((acc_mean, acc_min, acc_max, acc_var))

In [ ]:
rec_acc = [(rec, np.array(vals).mean(axis=0)) for rec, vals in acc_vals.items()]
rec_acc = sorted(rec_acc, key=lambda x: x[0])
for rec, vals in rec_acc:
    print("rec={:.5f}: mean={:.4f} min={:.4f} max={:.4f} var={:.8f}".format(rec, vals[0], vals[1], vals[2], vals[3]))

results:
- reconstruction loss does not have a large impact on the acc
- when comparing just the best models, acc actually drops!

# Compare effect of BS on ACC

In [ ]:
# with weight decay vs without weight decay
acc_vals = {}
for config, stats in runs_success:
    all_accs = stats["valid"]["affnist"]["acc"][-30:]
    acc_mean = np.mean(all_accs)
    acc_min = np.min(all_accs)
    acc_max = np.max(all_accs)
    acc_var = np.var(all_accs)
    bs = config.train.batch_size
    if bs not in acc_vals:
        acc_vals[bs] = []
    acc_vals[bs].append((acc_mean, acc_min, acc_max, acc_var))

In [ ]:
bs_acc = [(bs, np.array(vals).mean(axis=0)) for bs, vals in acc_vals.items()]
bs_acc = sorted(bs_acc, key=lambda x: x[0])
for bs, vals in bs_acc:
    print("bs={:04d}: mean={:.4f} min={:.4f} max={:.4f} var={:.8f}".format(bs, vals[0], vals[1], vals[2], vals[3]))

In [ ]:
# with weight decay vs without weight decay
acc_vals = {}
for config, stats in runs_success:
    all_accs = stats["valid"]["affnist"]["acc"][-30:]
    if max(all_accs) < 0.80:
        continue
    acc_mean = np.mean(all_accs)
    acc_min = np.min(all_accs)
    acc_max = np.max(all_accs)
    acc_var = np.var(all_accs)
    bs = config.train.batch_size
    if bs not in acc_vals:
        acc_vals[bs] = []
    acc_vals[bs].append((acc_mean, acc_min, acc_max, acc_var))
print(len(acc_vals[bs]))

In [ ]:
bs_acc = [(bs, np.array(vals).mean(axis=0)) for bs, vals in acc_vals.items()]
bs_acc = sorted(bs_acc, key=lambda x: x[0])
for bs, vals in bs_acc:
    print("bs={:04d}: mean={:.4f} min={:.4f} max={:.4f} var={:.8f}".format(bs, vals[0], vals[1], vals[2], vals[3]))

# Compare effect of lr on ACC

In [ ]:
# with weight decay vs without weight decay
acc_vals = {}
for config, stats in runs_success:
    all_accs = stats["valid"]["affnist"]["acc"][-10:]
    if max(all_accs) < 0.7:
        continue
    acc_mean = np.mean(all_accs)
    acc_min = np.min(all_accs)
    acc_max = np.max(all_accs)
    acc_var = np.var(all_accs)
    lr = config.optimizer_args.lr
    if lr not in acc_vals:
        acc_vals[lr] = []
    acc_vals[lr].append((acc_mean, acc_min, acc_max, acc_var))

In [ ]:
lr_acc = [(lr, np.array(vals).mean(axis=0)) for lr, vals in acc_vals.items()]
lr_acc = sorted(lr_acc, key=lambda x: x[0])
for lr, vals in lr_acc:
    print("lr={:.5f}: mean={:.4f} min={:.4f} max={:.4f} var={:.8f}".format(lr, vals[0], vals[1], vals[2], vals[3]))

In [ ]:
# with weight decay vs without weight decay
acc_vals = {}
for config, stats in runs_success:
    all_accs = stats["valid"]["affnist"]["acc"][-10:]
    if max(all_accs) < 0.80:
        continue
    acc_mean = np.mean(all_accs)
    acc_min = np.min(all_accs)
    acc_max = np.max(all_accs)
    acc_var = np.var(all_accs)
    lr = config.optimizer_args.lr
    if lr not in acc_vals:
        acc_vals[lr] = []
    acc_vals[lr].append((acc_mean, acc_min, acc_max, acc_var))
print(len(acc_vals[lr]))

In [ ]:
lr_acc = [(lr, np.array(vals).mean(axis=0)) for lr, vals in acc_vals.items()]
lr_acc = sorted(lr_acc, key=lambda x: x[0])
for lr, vals in lr_acc:
    print("lr={:.5f}: mean={:.4f} min={:.4f} max={:.4f} var={:.8f}".format(lr, vals[0], vals[1], vals[2], vals[3]))

# Check reconstructions

In [ ]:
for config, stats in runs_success:
    all_accs = stats["valid"]["affnist"]["acc"]
    max_acc = max(all_accs)
    if max_acc < 0.85 or config.train.batch_size != 128 or config.optimizer_args.lr != 0.001:
        continue
    print("bs={:4d}, lr={:.5f}, wd={:.5f} rec_weights={:8.5f}  max_acc={:.4f}, n_epochs={}, {}".format(
            config.train.batch_size,
            config.optimizer_args.lr,
            config.optimizer_args.weight_decay,
            config.loss.rec.weight,
            max_acc,
            len(all_accs),
            stats["notes"]
             )
             )
    model_dir = config.names.model_dir
    print("tensorboard --logdir={}/logs".format(p_experiments / model_dir))

results:
- reconstructions were really bad for all of those with low rec weight

In [ ]:
# next find good reconstuctions to understand which factors impact the reconstuction quality!

# Correlation

In [ ]:
last_n_epochs = 20

data = []
# with weight decay vs without weight decay
acc_vals = {}
for config, stats in runs_success:
    accs_train = stats["train"]["acc"][-last_n_epochs:]
    accs_mn_valid = stats["valid"]["mnist"]["acc"][-last_n_epochs:]
    accs_an_valid = stats["valid"]["affnist"]["acc"][-last_n_epochs:]

    acc_train_mean = np.mean(accs_train)
    acc_valid_mn_mean = np.mean(accs_mn_valid)
    acc_valid_an_mean = np.mean(accs_an_valid)
    #
    lr = config.optimizer_args.lr
    bs = config.train.batch_size
    wd = config.optimizer_args.weight_decay
    rl = config.loss.rec.weight
    #
    data.append([lr, bs, wd, rl, acc_train_mean, acc_valid_mn_mean, acc_valid_an_mean])
    #acc_mean = np.mean(all_accs)
    #acc_min = np.min(all_accs)
    #acc_max = np.max(all_accs)
    #acc_var = np.var(all_accs)
    #bs = config.train.batch_size
    #if bs not in acc_vals:
    #    acc_vals[bs] = []
    #acc_vals[bs].append((acc_mean, acc_min, acc_max, acc_var))
data = np.array(data)
names = ["lr", "bs", "weight_decay", "rec_loss", "acc_train", "acc_mn", "acc_an"]

In [ ]:
print(data.shape)

In [ ]:
C = np.corrcoef(data.T)
plot_mat(C, row_names=names, col_names=names, scale_factor=1)